In [37]:
import pickle
import pandas as pd
import folium
import geopy
import geopandas
from tqdm import tqdm

In [24]:
from geopy import Nominatim
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

# import data
I need to drop the `name` column cause there are some names missing.

In [25]:
with open('../data/dict_booking.pickle', 'rb') as dict_booking:
    dict_booking = pickle.load(dict_booking)

In [26]:
print(len(dict_booking['url']), len(dict_booking['name']), len(dict_booking['pagina']), len(dict_booking['currentPrice']), len(dict_booking['address']))

1123 968 1123 1123 1123


In [27]:
dict_booking.keys()

dict_keys(['url', 'name', 'pagina', 'address', 'currentPrice', 'previousPrice', 'rating', 'reviewNumber', 'image'])

In [28]:
del dict_booking['name']

In [29]:
dict_booking.keys()

dict_keys(['url', 'pagina', 'address', 'currentPrice', 'previousPrice', 'rating', 'reviewNumber', 'image'])

In [30]:
df_booking = pd.DataFrame(dict_booking)
df_booking.head()

,url,pagina,address,currentPrice,previousPrice,rating,reviewNumber,image
0,https://www.booking.com/hotel/it/b-amp-b-duca-...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[via giuseppe zurria 6, 95121 catania, italy]",€ 566,€ 980,8.2,608 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...
1,https://www.booking.com/hotel/it/justbeandb.en...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[contrada san leonardo 36, 98060 piraino, italy]","€ 1,260",NaN,9.7,301 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...
2,https://www.booking.com/hotel/it/maison-ortigi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[piazza san giuseppe 25, ortigia, 96100 siracu...",€ 603,NaN,8.1,"2,798 reviews",https://cf.bstatic.com/xdata/images/hotel/squa...
3,https://www.booking.com/hotel/it/b-amp-b-una-t...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[via torretta sferracavallo, province of paler...",€ 581,NaN,8.9,294 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...
4,https://www.booking.com/hotel/it/alcantara-res...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[via colonnello francesco mannino, 98030 gaggi...","€ 1,134",NaN,9.0,116 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...


In [31]:
df_booking = df_booking.explode('address')
df_booking.head()

,url,pagina,address,currentPrice,previousPrice,rating,reviewNumber,image
0,https://www.booking.com/hotel/it/b-amp-b-duca-...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"via giuseppe zurria 6, 95121 catania, italy",€ 566,€ 980,8.2,608 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...
1,https://www.booking.com/hotel/it/justbeandb.en...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"contrada san leonardo 36, 98060 piraino, italy","€ 1,260",NaN,9.7,301 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...
2,https://www.booking.com/hotel/it/maison-ortigi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"piazza san giuseppe 25, ortigia, 96100 siracus...",€ 603,NaN,8.1,"2,798 reviews",https://cf.bstatic.com/xdata/images/hotel/squa...
2,https://www.booking.com/hotel/it/maison-ortigi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"located in siracusa in the sicily region, 200 ...",€ 603,NaN,8.1,"2,798 reviews",https://cf.bstatic.com/xdata/images/hotel/squa...
3,https://www.booking.com/hotel/it/b-amp-b-una-t...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"via torretta sferracavallo, province of palerm...",€ 581,NaN,8.9,294 reviews,https://cf.bstatic.com/xdata/images/hotel/squa...


In [32]:
print(df_booking.shape)
df_booking.drop_duplicates(inplace=True)
print(df_booking.shape)

(1241, 8)
(1241, 8)


checking dtypes

In [33]:
df_booking.dtypes

url              object
pagina           object
address          object
currentPrice     object
previousPrice    object
rating           object
reviewNumber     object
image            object
dtype: object

In [34]:
type(df_booking['address'][0])

str

# Geocoding

In [44]:
from geopy.extra.rate_limiter import RateLimiter
tqdm.pandas()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_booking['location'] = df_booking['address'].progress_apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_booking['point'] = df_booking['location'].progress_apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_booking[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_booking['point'].tolist(), index=df_booking.index)

  2%|▏         | 24/1241 [00:22<20:22,  1.00s/it]RateLimiter caught an error, retrying (0/2 tries). Called with (*('located in san vito lo capo, 350 yards from san vito lo capo beach, hermes hotel provides accommodation with a shared lounge, free private parking, a terrace and a bar. this 3-star hotel offers a concierge service and a tour desk. the accommodation features a 24-hour front desk, airport transfers, room service and free wifi throughout the property.',), **{}).
Traceback (most recent call last):
  File "c:\Users\annas\anaconda3\envs\iron\lib\site-packages\geopy\extra\rate_limiter.py", line 136, in _retries_gen
    yield i  # Run the function.
  File "c:\Users\annas\anaconda3\envs\iron\lib\site-packages\geopy\extra\rate_limiter.py", line 274, in __call__
    res = self.func(*args, **kwargs)
  File "c:\Users\annas\anaconda3\envs\iron\lib\site-packages\geopy\geocoders\arcgis.py", line 173, in geocode
    return self._authenticated_call_geocoder(url, callback, timeout=timeout)


In [45]:
map1 = folium.Map(
    location=[37.75120776952038, 14.992361985738308],
    tiles='cartodbpositron',
    zoom_start=12,
)
df_booking[df_booking['latitude'].isnull() == False].apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1